# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
columns = ['Postcode', 'Borough', 'Neighborhood']

Scraping table with BeautifulSoup

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class': 'wikitable sortable'})


In [4]:
with open('zipcodes.csv', 'w') as f:
    
    for tr in table.find_all('tr'):
        
        tds = tr.find_all('td')
        if not tds:
            continue
        
        a, b, c = [td.text.strip() for td in tds[:4]]
        
        print(', '.join([a, b, c]), file=f)

In [5]:
df = pd.read_csv('zipcodes.csv')
df.columns = columns

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df = df[ df['Borough'] != ' Not assigned' ]

More than one neighborhood can exist in one postal code area. 

For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [7]:
new = []
for i in range(df.shape[0]-1):
    if df.iloc[i,0] == df.iloc[i+1, 0]:
        df.iloc[i,2] = df.iloc[i,2] + ', ' + df.iloc[i+1, 2]
        
df.drop_duplicates(subset='Postcode', keep='first', inplace=True)
df.reset_index(inplace=True, drop=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
df['Neighborhood'][ df['Neighborhood'] == ' Not assigned' ] = df['Borough']

In [9]:
df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df.shape

(103, 3)

In [11]:
df.to_csv('toronto.csv', index=False)